## Наследование. Множественное наследование. MRO. Композиция. Делегирование. Псевдозакрытые атрибуты классов

### Множественное наследование и MRO

Множественное наследование (подмешивание классов) - когда класс-ребенок имеет больше одного класса-родителя. В таком случае наследуются все методы и атрибуты суперклассов. 

In [96]:
class Employee:
    '''Общий класс-родитель (дедушка)'''
  def __init__(self, name, surname):
    self.name = name
    self.surname = surname
    self.__salary = 350  # _Employee__salary
    self.bankaccount = 0

  def work(self, *args):
    raise NotImplementedError

class Linguist(Employee):
    '''Класс-мама'''
  def __init__(self, name, surname):
    Employee.__init__(self, name, surname)
    self.__salary = self._Employee__salary * 1.5
    self.publications = []
    
  def work(self, hours, name_of_publ):
    print('Working...')
    sleep(hours)
    self.bankaccount += self.salary * hours
    self.publications.append(name_of_publ)
    print(f'{self.name} {self.surname} has published a paper {name_of_publ} in Voprosy Yazykoznaniya')

  def readlingpapers(self, lingpaper):
    print('Reading...')
    sleep(len(lingpaper) // 10)
    print(f'{self.name} {self.surname} has read {lingpaper}')

class Programmer(Employee):
    '''Класс-папа'''
  def __init__(self, name, surname):
    Employee.__init__(self, name, surname)
    self.__salary *= self._Employee__salary * 2.5
    self.projects = []

  def work(self, hours, name_of_project):
    print('Working...')
    sleep(hours)
    self.bankaccount += self.salary * hours
    self.projects.append(name_of_project)
    print(f'{self.name} {self.surname} has committed a repo {name_of_project} to GitHub')

  def learnpython(self, hours):
    print('Studying...')
    sleep(hours)
    print(f'{self.name} {self.surname} has learned a bit of Python')

In [82]:
class ComputerLinguist(Linguist, Programmer):
    '''Наследуемся одновременно от лингвиста и от программиста'''
  def __init__(self, name, surname):
    Linguist.__init__(self, name, surname)  # явным образом вызываем init класса "лингвист"
    Programmer.__init__(self, name, surname)
    self.salary *= 0.9

  def work(self, hours, name_of_paper_with_code):
    print('Working...')
    sleep(hours)
    self.publications.append(name_of_paper_with_code)
    self.projects.append(name_of_paper_with_code)
    self.bankaccount += self.salary * hours
    print(f'{self.name} {self.surname} has published {name_of_paper_with_code} in ACL Papers')

При таком раскладе, если мы вдруг не определили бы у класса-ребенка метод work, а у обоих классов-родителей он был, питон пошел бы искать у того класса, который мы в скобках написали первым. 

![Image](https://media.geeksforgeeks.org/wp-content/uploads/220px-diamond_inheritance-svg.png)

Эта вещь и называется Method Resolution Order = MRO. Когда мы вызываем какой-нибудь метод класса, питон попросту ищет такой метод в пространстве имен сперва самого этого класса, потом, если не находит, то у класса-родителя, от которого наследуем первым, а если у него нет, то у его родителя, и так далее. 

Что касается динамических атрибутов, то если есть одноименные, то все решает порядок вызовов init: кого последним вызвали, того и зарплата. 

### Композиция и делегирование

Другой способ сочетать классы при проектировании программы - композиция. Это когда мы вкладываем экземпляры классов в другие классы и вызываем их методы внутри определения класса. 

In [65]:
class Employee:
    '''Класс общего назначения'''
  def __init__(self, name, surname):
    self.name = name
    self.surname = surname
    self.salary = 350
    self.bankaccount = 0

  def work(self, *args):
    raise NotImplementedError

  def sleep(self, message):
    print(message)

class Linguist:
  def __init__(self, name, surname, salary_coef):
    self.employee = Employee(name, surname)  # создаем экземпляр класса Employee и складываем его в атрибут класса Лингвист. 
    self.employee.salary *= salary_coef  # у самого лингвиста при этом нет атрибутов, кроме employee, поэтому если нам нужна зарплата, обращаемся напрямую к атрибутам employee
  
  def work(self, hours):
    print('Working...')
    sleep(hours)
    self.employee.bankaccount += hours * self.employee.salary

  def sleep(self, message):
    self.employee.sleep(message)  # делегирование

  def __getattr__(self, attr):
    '''А чтобы можно было обращаться к атрибутам вложенного класса, приходится делегировать'''
    print(f'{attr} is called')
    return getattr(self.employee, attr) # встроенная функция 

  def __setattr__(self, attr, value):
    print(f'trying to set attr {attr}')
    self.__dict__[attr] = value

Магические методы \_\_getattr\_\_ и \_\_setattr\_\_ вызываются, когда мы обращаемся к неопределенному (не существующему) атрибуту через точку. То есть, когда они вызываются в этом конкретном случае: у класса Linguist есть только один атрибут - employee. Если мы к нему обратимся, эти методы не сработают. Но если мы обратимся к атрибуту name, которого у Лингвиста нет, потому что он есть только у вложенного экземпляра класса Employee, то вызовется метод \_\_getattr\_\_, а тут-то мы и перенаправим на атрибут вложенного экземпляра. 

Это и называется делегирование. В примере выше оно происходит в методах sleep, getattr & setattr.

Почему нам приходится таким странным образом переопределять setattr? Потому что если мы напишем просто 

    def __setattr__(self, attr, value):
        self.attr = value
        
Возникнет рекурсия: точка опять вызовет этот же метод. Кстати говоря, зациклить можно и getattr, если переопределить его для получения атрибутов этого же класса. 

Поэтому мы идем обходным путем и используем встроенный атрибут класса \_\_dict\_\_. Что это за атрибуты такие? Это специальные зарезервированные атрибуты, некоторые из них по умолчанию у любого класса появляются, некоторые обычно появляются, но могут нет (\_\_dict\_\_ как раз может не быть). Этот самый \_\_dict\_\_ - это действительно всего лишь обычнейший словарь, где в качестве ключей хранятся имена атрибутов, а в значениях, собственно, их значения. 

У класса есть еще парочка полезных атрибутов, например, \_\_class\_\_ возвращает ссылку из экземпляра на класс. В этой ссылке содержится имя класса \_\_name\_\_ и последовательность \_\_bases\_\_, в которой лежат классы-родители. 

In [73]:
class Employee:
  def __init__(self, name, surname):
    self.name = name
    self.surname = surname
    self.salary = 350
    self.bankaccount = 0

  def work(self, *args):
    raise NotImplementedError

  def __repr__(self):
    return f"{self.__class__.__name__}('{self.name}', '{self.surname}')"  # так с помощью этих атрибутов можно определить метод repr, чтобы меньше хардкодить

In [ ]:
class ComputerLinguist(Linguist, Programmer):
    '''Наследуемся одновременно от лингвиста и от программиста'''
  def __init__(self, name, surname):
    for base in self.__class__.__bases__:
        base.__init__(self, name, surname)
    # Альтернатива закомментированному коду
    # Linguist.__init__(self, name, surname)  # явным образом вызываем init класса "лингвист"
    # Programmer.__init__(self, name, surname)
    self.salary *= 0.9

Композиция выглядит более громоздкой и сложной, чем наследование, но у нее есть свои области применения. Обычно композиция + делегирование используется, когда нам нужен какой-нибудь класс-контролер, который сам по себе ничего не делает, но, например, следит за состоянием другого класса. Можно логировать (выводить логи про каждый пшик), например, если в переопределенных методах getattr & setattr прописать какие-нибудь принты, можно отслеживать обращения к атрибутам. 

Композиция также позволяет агрегировать экземпляры другого класса:

In [76]:
class ARD:
    '''отдел Advanced Research & Development '''
  def __init__(self, *args):
    self.members = list(args)  # куча лингвистов

  def add_member(self, linguist):
    self.members.append(linguist)

  def raise_salary(self, coef):
    '''массово всему отделу повышаем зарплатку'''
    for linguist in self.members:
      linguist.salary *= coef

  def workEveryone(self, hour):
    '''или гоним работать лентяев'''
    for linguist in self.members:
      linguist.work(hour)

Фишку с делегированием и переопределением getattr & setattr также ограниченно можно использовать для реализации инкапсуляции: закрыть доступ к атрибутам класса. 

Ну и вот пример класса-контроллера, который сам по себе ничего не делает, только следит за работой внутреннего класса:

In [80]:
class Wrapper:
  def __init__(self, object):
    self.wrapped = object
  
  def __getattr__(self, attr):
    print(f'Big brother is watching you, here is your {attr}')
    return getattr(self.wrapped, attr)

In [86]:
w = Wrapper(gorbunova)

In [87]:
w.name

Big brother is watching you, here is your name


'Ira'

Похожим образом работают дескрипторы классов и декораторы. 

### Псевдозакрытые атрибуты классов

Напомню, в ООП есть такая концепция, как инкапсуляция: она о том, что мы должны иметь доступ к классу только через его интерфейс, а его внутренности должны быть от нас скрыты. Например, в объектно-ориентированном языке Java это реализуется с помощью операторов public & private:

    public class tokenisation {
        public static void main(String[] args) throws Exception
        {
            System.setProperty("file.encoding", "UTF-8");
            Scanner scan = new Scanner(System.in, "UTF-8");
            final DecimalFormat df = new DecimalFormat("0.00");

            System.out.println("Enter file path:");
            String path = scan.nextLine();
            File file = new File(path);

            ArrayList<String> text = new ArrayList<String>();

            Pattern pattern = Pattern.compile("[a-zА-Яа-яёЁ]+(-[a-zА-Яа-яёЁ]+)*", Pattern.CASE_INSENSITIVE);

            BufferedReader br = new BufferedReader(new FileReader(file));
            String str;
            while ((str = br.readLine()) != null) {
                Matcher matcher = pattern.matcher(str);
                while (matcher.find()) {
                    String match = matcher.group();
                    text.add(match);
                }

            }
            br.close();
            scan.close();

            Set<String> unique = new HashSet<String>(text);

            float lexdiv = (float)unique.size() / (float)text.size() * 100;

            System.out.println(df.format(lexdiv));

        }
    }
    
Так выглядит код на этом языке. Можно заметить, что здесь встречается словечко public, которое сообщает, что этот класс и эту функцию можно вызывать из других мест. Если бы было private, то мы не имели бы доступа к этому объекту извне. 

У питона таких готовых инструментов нет, но можно закрывать атрибуты и имитировать инкапсуляцию разными способами. В частности, в питоне есть псевдозакрытые атрибуты: это скорее про конвенции, чем про синтаксис, но тем не менее. 

Если мы хотим сделать какой-то атрибут класса закрытым, то мы должны назвать его с двойным нижним подчеркиванием перед именем:

    self.__salary
    
Тогда внутри класса мы можем к нему обращаться по этому имени, а вне тоже вообще-то можем, но тогда придется писать \_Linguist\_\_salary, потому что такие атрибуты интерпретатор автоматически переименовывает при определении класса. 